# Jaime Ariel Chitay
# 19002065
# Examen Final

## Importación de librerias

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [40]:
data = pd.read_csv('data_titanic_proyecto.csv')


#### Variables a usar: 

1) Age <br>
2) Fare, <br>
3) Embarked, <br>
4) passenger_class, <br>
5) passenger_sex y <br>
6) passenger_survived (que en realidad es la variable objetivo "y").</p>

In [3]:
data.describe()

,PassengerId,Age,SibSp,Parch,Fare
count,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,29.699118,0.523008,0.381594,32.204208
std,257.353842,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,38.000000,1.000000,0.000000,31.000000
max,891.000000,80.000000,8.000000,6.000000,512.329200


In [4]:
data['Embarked'].mode()

0    S
dtype: object

Como se puede observar en la info, Age tiene varios valores nan y Embarked cuenta con únicamente con 2. Para tratar el problema, se limpiará la variable Age llenando los espacios con la mediana y para la variable Embarked se llenarán los espacios utilizando la moda (S = Southampton).

# Importación de Dataset

In [5]:
dataset = data.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis = 1)

In [6]:
cleanAge = dataset['Age'].fillna(dataset['Age'].median()) #Limpiar nan de Age, llenando con la mediana. 
dataset = dataset.drop(['Age'], axis=1)
dataset['Age'] = cleanAge

In [7]:
cleanEmbarked = dataset['Embarked'].astype(object).fillna('S') #Limpiar Embarked llenando con la moda. 
dataset = dataset.drop(['Embarked'], axis=1)
dataset['Embarked'] = cleanEmbarked

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
Fare                  891 non-null float64
passenger_class       891 non-null object
passenger_sex         891 non-null object
passenger_survived    891 non-null object
Age                   891 non-null float64
Embarked              891 non-null object
dtypes: float64(2), object(4)
memory usage: 41.8+ KB


En este paso se asignan valores numéricos a las variables categóricas a través de Label Encoder:

In [9]:
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(dataset['Embarked'])
embarked = label_encoder.transform(dataset['Embarked'])
label_encoder.fit(dataset['passenger_sex'])
sex = label_encoder.transform(dataset['passenger_sex'])
label_encoder.fit(dataset['passenger_class'])
pclass = label_encoder.transform(dataset['passenger_class'])
label_encoder.fit(dataset['passenger_survived'])
psurvived = label_encoder.transform(dataset['passenger_survived'])

In [37]:
dataset = dataset.drop(['passenger_class','passenger_survived','Embarked', 'passenger_sex'], axis = 1)
dataset['Embarked'] = embarked
dataset['passenger_sex'] = sex
dataset['passenger_class'] = pclass
dataset['passenger_survived'] = psurvived

# Normalización de datos

In [39]:
from sklearn.preprocessing import normalize
features=dataset.iloc[:,0:2]
normalize=preprocessing.normalize(features)

In [12]:
Farenorm=normalize[:,0]
Agenorm=normalize[:,1]
dataset['Fare']=Farenorm
dataset['Age']=Agenorm
dataset.head()

,Fare,Age,Embarked,passenger_sex,passenger_class,passenger_survived
0,0.312988,0.949757,2,1,0,0
1,0.882444,0.470417,0,0,2,1
2,0.291564,0.956551,2,0,0,1
3,0.834942,0.550338,2,0,2,1
4,0.224148,0.974555,2,1,0,0


# Datos de entrenamiento y validación

In [13]:
y = dataset['passenger_survived'] 
y = pd.DataFrame(y,columns=['passenger_survived'])#Split data 
x = dataset.drop(['passenger_survived'],axis=1)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)
#entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [15]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(569, 5)
(143, 5)
(179, 5)
(569, 1)
(143, 1)
(179, 1)


# Cálculo de métricas

In [16]:
# Se importan las funciones que provee sklearn para el cálculo de métricas:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

# Decision tree

In [17]:
from sklearn import tree
def decisiontree(md,X,Y):
    clasificador_gini = tree.DecisionTreeClassifier(criterion = 'gini', max_depth=md)
    clasificador_gini.fit(X_train, y_train)
    yhat_gini = clasificador_gini.predict(X)
    accuracy = accuracy_score(Y,yhat_gini)
    error = 1 - accuracy
    reporte = classification_report(Y, yhat_gini)
    f1 = f1_score(Y,yhat_gini)
    precision = precision_score(Y,yhat_gini)
    recall = recall_score(Y,yhat_gini)
    metricas_array = np.array([accuracy,error,precision,recall,f1])
    df_metricas=pd.DataFrame(metricas_array,index=['Accuracy','Error','Precision','Recall','F1'])
    lista=[df_metricas,yhat_gini]
    return lista


In [18]:
dtreetrain1=decisiontree(3,X_train,y_train)
dtreeval1=decisiontree(3,X_val,y_val)
dtreeval2=decisiontree(5,X_val,y_val)

In [19]:
dt = pd.concat([dtreetrain1[0],dtreeval1[0],dtreeval2[0]],axis=1).T
dt['Cadena de configuración'] = 'D Tree train,md=3','D Tree val,md=3','D Tree val,md=5'
dt['Modelo'] = 'Decision tree','Decision tree','Decision tree'
dt = dt[['Modelo','Cadena de configuración','Accuracy','Error','Precision','Recall','F1']]
dt

,Modelo,Cadena de configuración,Accuracy,Error,Precision,Recall,F1
0,Decision tree,"D Tree train,md=3",0.841828,0.158172,0.904459,0.654378,0.759358
0,Decision tree,"D Tree val,md=3",0.797203,0.202797,0.828571,0.557692,0.666667
0,Decision tree,"D Tree val,md=5",0.783217,0.216783,0.769231,0.576923,0.659341


# SVM

In [20]:
from sklearn.svm import SVC
def svm(X,Y,c):
    svm_titanic = SVC(kernel='linear',C=c)
    svm_titanic.fit(X_train, y_train['passenger_survived'])
    y_hat_svm = svm_titanic.predict(X)
    svm_accuracy = accuracy_score(Y, y_hat_svm)
    svm_error = 1 - svm_accuracy
    svm_precision =  precision_score(Y,y_hat_svm)
    svm_recall = recall_score(Y,y_hat_svm)
    svm_f1 = f1_score(Y,y_hat_svm)
    metricas_svm = np.array([svm_accuracy,svm_error,svm_precision,svm_recall,svm_f1])
    df_metricas=pd.DataFrame(metricas_svm,index=['Accuracy','Error','Precision','Recall','F1'])
    listasvm=[df_metricas,y_hat_svm]
    return listasvm

In [21]:
svmtrain1=svm(X_train,y_train,0.8)
svmval1=svm(X_val,y_val,0.8)
sv = pd.concat([svmtrain1[0],svmval1[0]],axis=1).T
sv['Cadena de configuración'] = 'SVM train1,C=0.8','SVM val1,C=0.8'
sv['Modelo'] = 'SVM','SVM'
sv = sv[['Modelo','Cadena de configuración','Accuracy','Error','Precision','Recall','F1']]
sv

,Modelo,Cadena de configuración,Accuracy,Error,Precision,Recall,F1
0,SVM,"SVM train1,C=0.8",0.799649,0.200351,0.758794,0.695853,0.725962
0,SVM,"SVM val1,C=0.8",0.748252,0.251748,0.660000,0.634615,0.647059


# Regresión Logística con TensorFlow

In [22]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

(569, 5)
(143, 5)
(179, 5)
(569, 1)
(143, 1)
(179, 1)


In [23]:
def yrl(y): ##Esta se utiliza para convertir los pandas series a arrays para luego poder obtener las metricas
    ydata = y.values
    return ydata

In [24]:
ytrainrl=yrl(y_train)
yvalrl=yrl(y_val)
ytestrl=yrl(y_test)

%store ytrainrl
%store yvalrl
%store ytestrl

Stored 'ytrainrl' (ndarray)
Stored 'yvalrl' (ndarray)
Stored 'ytestrl' (ndarray)


In [25]:
def regresion_logistica(x,y,lr,fr,epoch,batch):
    session = tf.Session()
    X = tf.placeholder(shape=[None, 5], dtype = tf.float32)
    Y = tf.placeholder(shape = [None, 1], dtype=tf.float32)
    W = tf.Variable(tf.random_normal(shape=[5,1]))
    b = tf.Variable(tf.random_normal(shape=[1,1]))
    y_hat = tf.add(tf.matmul(X, W), b)
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, labels = Y)) + fr*tf.nn.l2_loss(W)
    init = tf.global_variables_initializer()
    session.run(init)
    optimizador = tf.train.GradientDescentOptimizer(learning_rate=lr)
    opt_minimize = optimizador.minimize(cost)
    salida_prob = tf.sigmoid(y_hat)
    clasificacion_lr = tf.round(tf.sigmoid(y_hat))
    clasificacion_final = tf.cast(tf.equal(clasificacion_lr, Y), tf.float32)
    accuracy = tf.reduce_mean(clasificacion_final)
    epochs = epoch
    batch_size = batch
    for epoch in range(epochs):
        batches = int(X_train.shape[0] / batch_size)
        for i in range(batches):
            session.run(opt_minimize, feed_dict={X: X_train, Y: y_train})
            parametrosW = session.run(W,feed_dict={X: X_train, Y: y_train})
            parametrosb = session.run(b,feed_dict={X: X_train, Y: y_train}) #Hasta acá se entrena
            prob_output = session.run(salida_prob,feed_dict={X: x, Y: y})
            cc = session.run(clasificacion_final,feed_dict={X: x, Y: y})
            acc = session.run(accuracy, feed_dict={X: x, Y: y})
            errorlr = 1 - acc
            metricas_arraylr = np.array([acc,errorlr]) #accuracy en index 0; error en index 1.
            listlr=[metricas_arraylr,cc,prob_output,parametrosW,parametrosb]
            
    return listlr

In [26]:
rltrainpred1 = regresion_logistica(X_train,ytrainrl,0.01,0.01,200,30)[1]
rltrainW = regresion_logistica(X_train,ytrainrl,0.01,0.01,200,30)[3]
rltrainb = regresion_logistica(X_train,ytrainrl,0.01,0.01,200,30)[4]
rltrainmetric = regresion_logistica(X_train,ytrainrl,0.01,0.01,200,30)[0]

In [27]:
rltrainpred2 = regresion_logistica(X_train,ytrainrl,0.001,10,200,30)[1]
rltrainW2 = regresion_logistica(X_train,ytrainrl,0.001,10,200,30)[3]
rltrainb2 = regresion_logistica(X_train,ytrainrl,0.001,10,200,30)[4]
rltrainmetric2 = regresion_logistica(X_train,ytrainrl,0.001,10,200,30)[0]

In [28]:
rltrainpred3 = regresion_logistica(X_train,ytrainrl,0.0001,5,200,30)[1]
rltrainW3 = regresion_logistica(X_train,ytrainrl,0.0001,5,200,30)[3]
rltrainb3 = regresion_logistica(X_train,ytrainrl,0.0001,5,200,30)[4]
rltrainmetric3 = regresion_logistica(X_train,ytrainrl,0.0001,5,200,30)[0]

In [29]:
rltrainpred4 = regresion_logistica(X_train,ytrainrl,0.0001,0.01,200,30)[1]
rltrainW4 = regresion_logistica(X_train,ytrainrl,0.0001,0.01,200,30)[3]
rltrainb4 = regresion_logistica(X_train,ytrainrl,0.0001,0.01,200,30)[4]
rltrainmetric4 = regresion_logistica(X_train,ytrainrl,0.0001,0.01,200,30)[0]

In [31]:
rlvalpred1 = regresion_logistica(X_val,yvalrl,0.0001,0.01,200,30)[1]

In [32]:
#funcion similar a los modelos anteriores: regresa df con metricas, yhats y la salida probabilistica (para este caso)
def rlfinal(y,yhat):
    prediccion = yhat
    accuracy = accuracy_score(y,prediccion.astype(int))  #las y que se ingresan son las obtenidas en la funcion
    error = 1 - accuracy
    precision = precision_score(y,prediccion.astype(int))
    recall = recall_score(y,prediccion.astype(int))
    f1 = f1_score(y,prediccion.astype(int))
    metricas_rl = np.array([accuracy,error,precision,recall, f1])
    df_metricasrl=pd.DataFrame(metricas_rl,index=['Accuracy','Error','Precision','Recall','F1'])
    listarl=[df_metricasrl,prediccion.astype(int).T]
    return listarl

In [33]:
rltrain1 = rlfinal(ytrainrl,rltrainpred1)
rltrain2 = rlfinal(ytrainrl,rltrainpred2)
rltrain3 = rlfinal(ytrainrl,rltrainpred3)
rltrain4 = rlfinal(ytrainrl,rltrainpred4)
rlval = rlfinal(yvalrl,rlvalpred1)

In [34]:
rl = pd.concat([rltrain1[0],rltrain2[0],rltrain3[0],rltrain4[0],rlval[0]],axis=1).T
rl['Cadena de configuración'] = 'rl_train,lr=0.01,fr=0.01,epoch=200,batch=30','rl_train,lr=0.001,fr=10,epoch=200,batch=30','rl_train,lr=0.0001,fr=5,epoch=200,batch=30','rl_train,lr=0.0001,fr=0.01,epoch=200,batch=30','rl_val,lr=0.0001,fr=0.01,epoch=200,batch=30'
rl['Modelo'] = 'RL','RL','RL','RL','RL'
rl = rl[['Modelo','Cadena de configuración','Accuracy','Error','Precision','Recall','F1']]
rl

,Modelo,Cadena de configuración,Accuracy,Error,Precision,Recall,F1
0,RL,"rl_train,lr=0.01,fr=0.01,epoch=200,batch=30",0.337434,0.662566,0.325328,0.686636,0.441481
0,RL,"rl_train,lr=0.001,fr=10,epoch=200,batch=30",0.000000,1.000000,0.000000,0.000000,0.000000
0,RL,"rl_train,lr=0.0001,fr=5,epoch=200,batch=30",1.000000,0.000000,1.000000,1.000000,1.000000
0,RL,"rl_train,lr=0.0001,fr=0.01,epoch=200,batch=30",0.066784,0.933216,0.038235,0.059908,0.046679
0,RL,"rl_val,lr=0.0001,fr=0.01,epoch=200,batch=30",1.000000,0.000000,1.000000,1.000000,1.000000


In [35]:
tabla1=pd.concat([dt,sv],axis=0)
tabla1

,Modelo,Cadena de configuración,Accuracy,Error,Precision,Recall,F1
0,Decision tree,"D Tree train,md=3",0.841828,0.158172,0.904459,0.654378,0.759358
0,Decision tree,"D Tree val,md=3",0.797203,0.202797,0.828571,0.557692,0.666667
0,Decision tree,"D Tree val,md=5",0.783217,0.216783,0.769231,0.576923,0.659341
0,SVM,"SVM train1,C=0.8",0.799649,0.200351,0.758794,0.695853,0.725962
0,SVM,"SVM val1,C=0.8",0.748252,0.251748,0.660000,0.634615,0.647059


# Fase de predicción
